In [1]:
import tensorflow as tf
import numpy as np

# Generate synthetic weather data
num_samples = 1000
sequence_length = 10
input_dim = 5  # Temperature, humidity, wind speed, etc.

# Generate random synthetic data
synthetic_data = np.random.rand(num_samples, sequence_length, input_dim)

# Split the data into inputs and outputs
X = synthetic_data[:, :-1]  # Input data (sequence_length - 1 time steps)
y = synthetic_data[:, -1]   # Output data (1 time step ahead)

# Define the Multi-Scale Multi-Head Attention model
class MultiScaleMultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, d_model, num_scales, name="multi_scale_multi_head_attention"):
        super(MultiScaleMultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model
        self.num_scales = num_scales
        self.head_size = d_model // num_heads

        assert self.head_size * num_heads == d_model, "d_model must be divisible by num_heads"

        # Define multi-head attention layers for each scale
        self.attention_layers = [tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=self.head_size,
            value_dim=self.head_size,
        ) for _ in range(num_scales)]

    def call(self, inputs):
        # Apply multi-head attention for each scale
        attention_outputs = []
        for i in range(self.num_scales):
            attention_output = self.attention_layers[i](inputs, inputs)
            attention_outputs.append(attention_output)

        # Concatenate attention outputs from different scales
        concatenated_attention = tf.concat(attention_outputs, axis=-1)

        return concatenated_attention

# Create a simple forecasting model using Multi-Scale Multi-Head Attention
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(sequence_length - 1, input_dim)),
    MultiScaleMultiHeadAttention(num_heads=4, d_model=32, num_scales=3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1)  # Output layer for forecasting
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Perform weather forecasting
new_input_data = np.random.rand(1, sequence_length - 1, input_dim)  # Replace with real input data
forecast = model.predict(new_input_data)

print("Weather Forecast:", forecast)


2023-09-04 11:33:25.429981: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10


2023-09-04 11:33:26.577079: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


32/32 [==============================] - 2s 25ms/step - loss: 0.1032
Epoch 2/10
32/32 [==============================] - 1s 22ms/step - loss: 0.0852
Epoch 3/10
32/32 [==============================] - 1s 17ms/step - loss: 0.0846
Epoch 4/10
32/32 [==============================] - 0s 14ms/step - loss: 0.0842
Epoch 5/10
32/32 [==============================] - 1s 16ms/step - loss: 0.0843
Epoch 6/10
32/32 [==============================] - 1s 19ms/step - loss: 0.0838
Epoch 7/10
32/32 [==============================] - 1s 26ms/step - loss: 0.0837
Epoch 8/10
32/32 [==============================] - 1s 20ms/step - loss: 0.0841
Epoch 9/10
32/32 [==============================] - 1s 16ms/step - loss: 0.0843
Epoch 10/10
1/1 [==============================] - 0s 197ms/step
Weather Forecast: [[0.47006938]]
